In [ ]:
%config InlineBackend.figure_format='retina'
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

rc('figure',figsize=(20,12))
rc('font',size=12)

import healpy as hp
from importlib import reload
import pysm3 as pysm

from qubicpack.utilities import Qubic_DataDir
import qubic
from qubic import NamasterLib
from qubic import QubicSkySim as qss
from qubic import camb_interface as qc

print('hello')


# Generate full-sky maps

In [ ]:
d = qubic.qubicdict.qubicDict()
d.read_from_file('likelihood_r_tests.dict')
d['nf_sub'] = 3
q = qubic.QubicInstrument(d)

gal_center = qubic.equ2gal(d['RA_center'], d['DEC_center'])
print(d['RA_center'], d['DEC_center'])
print(gal_center)

### Calculate CMB spectra with CAMB (through the QubicSkySim wrapper, or any other way)
ell, totDL, unlensedDL = qc.get_camb_Dl(r=0.1, lmax=3*d['nside'])



### Generate the maps
seed = np.random.randint(0,1e6)
cmb_dict = {'CAMBSpectra':totDL, 'ell':ell, 'seed':seed}
sky_config_cmb = {'cmb': cmb_dict}                ### CMB Only
sky_config_dust = {'dust':'d1'}                   ### Dust
sky_config = {'cmb': cmb_dict, 'dust':'d1'}       ### CMB + Dust


### Generate the maps at each sub-frequency
seed = np.random.randint(0, 1e6)
maps_cmb_full = qss.Qubic_sky(sky_config_cmb, d).get_simple_sky_map()
# maps_dust_full = qss.Qubic_sky(sky_config_dust, d).get_simple_sky_map()
# maps_full = qss.Qubic_sky(sky_config, d).get_simple_sky_map()

beams = qss.Qubic_sky(sky_config_cmb, d).instrument['beams']

Now display them

In [ ]:
### Display all maps
stn = ['I','Q','U']
rng = (np.std(maps_cmb_full, axis=(0,1))*3).astype(int)
rng[1:] = np.max(rng[1:])

figure()
numsub = 1
for i in range(d['nf_sub']):
    for istokes in [0,1,2]:
        hp.mollview(maps_cmb_full[i,:,istokes], min=-rng[istokes], max=rng[istokes],
                    sub=(d['nf_sub'],3,numsub), title=stn[istokes]+' subfreq {}'.format(i))
        numsub += 1
suptitle('CMB', y=1.02)
tight_layout()

# figure()
# numsub = 1
# for i in range(d['nf_sub']):
#     for istokes in [0,1,2]:
#         hp.mollview(maps_dust_full[i,:,istokes], min=-rng[istokes], max=rng[istokes],
#                     sub=(d['nf_sub'],3,numsub), title=stn[istokes]+' subfreq {}'.format(i))
#         numsub += 1
# suptitle('Dust', y=1.02)
# tight_layout()

# figure()
# numsub = 1
# for i in range(d['nf_sub']):
#     for istokes in [0,1,2]:
#         hp.mollview(maps_full[i,:,istokes], min=-rng[istokes], max=rng[istokes],
#                     sub=(d['nf_sub'],3,numsub), title=stn[istokes]+' subfreq {}'.format(i))
#         numsub += 1
# suptitle('CMB + Dust', y=1.02)
# tight_layout()


In [ ]:
# Spectra for the full sky
pnames = ['TT', 'EE', 'BB', 'TE']

cl_cmb_full = np.zeros((d['nf_sub'], 2*d['nside']+1, 6))
cl_dust_full = np.zeros((d['nf_sub'], 2*d['nside']+1, 6))
cl_full = np.zeros((d['nf_sub'], 2*d['nside']+1, 6))
for i in range(d['nf_sub']): 
    cl_cmb_full[i,:,:] = hp.anafast(maps_cmb_full[i,:,:].T, lmax=2*d['nside']).T
    # cl_dust_full[i,:,:] = hp.anafast(maps_dust_full[i,:,:].T, lmax=2*d['nside']).T
    # cl_full[i,:,:] = hp.anafast(maps_full[i,:,:].T, lmax=2*d['nside']).T
ll = np.arange(2*d['nside']+1)

fact = ll * (ll+1) / 2 / np.pi
isp = 2
for i in range(d['nf_sub']):
    subplot(2,d['nf_sub'],i+1)
    plot(ll, fact * cl_cmb_full[i,:, isp], 'r', label='CMB')
    # plot(ll, fact * cl_dust_full[i,:, isp], 'b', label='Dust')
    # plot(ll, fact * cl_full[i,:, isp], 'g', label='CMB + Dust')
    plot(ell, totDL[:,isp], label='Input CMB', alpha=0.5)
    yscale('log')
    ylim(1e-4,1e2)
    xlim(0,2*d['nside'])
    grid(alpha=0.4)
    xlabel('$\ell$')
    ylabel('$D_\ell$'+pnames[isp]+'({})'.format(i))
    title('Full-Sky Band={}'.format(i))
    if i==0:
        legend()
tight_layout()



# QUBIC Coverage

In [ ]:
# Very simplistic version: uniform coverage within 20 degrees aperture (to be improved for a more realistic coverage)
my_center = gal_center
ang_radius = 20. #degrees

mask = np.zeros(12*d['nside']**2)
v_center = np.array(hp.ang2vec(gal_center[0], gal_center[1], lonlat=True))
v_all = np.array(hp.pix2vec(d['nside'], np.arange(12*d['nside']**2)))
dang = np.degrees(np.arccos(np.dot(v_all.T, v_center)))
okpix = dang <= ang_radius

mask[okpix] = 1
sky_frac = np.sum(okpix) / len(okpix)
print(sky_frac)

hp.mollview(mask)

maps_cmb = np.zeros_like(maps_cmb_full)
maps_dust = np.zeros_like(maps_dust_full)
maps = np.zeros_like(maps_full)

for i in range(d['nf_sub']):
    maps_cmb[i,:,:] = (maps_cmb_full[i,:,:].T * mask).T
    # maps_dust[i,:,:] = (maps_dust_full[i,:,:].T * mask).T
    # maps[i,:,:] = (maps_full[i,:,:].T * mask).T


In [ ]:
rng = (np.std(maps_cmb, axis=(0,1))*10).astype(int)
rng[1:] = np.max(rng[1:])

reso = 13

figure()
numsub = 1
for i in range(d['nf_sub']):
    for istokes in [0,1,2]:
        hp.gnomview(maps_cmb[i,:,istokes], min=-rng[istokes], max=rng[istokes], rot=gal_center, reso=reso,
                    sub=(d['nf_sub'],3,numsub), title=stn[istokes]+' subfreq {}'.format(i))
        numsub += 1
suptitle('CMB', y=1.02)

# figure()
# numsub = 1
# for i in range(d['nf_sub']):
#     for istokes in [0,1,2]:
#         hp.gnomview(maps_dust[i,:,istokes], min=-rng[istokes], max=rng[istokes], rot=gal_center, reso=reso,
#                     sub=(d['nf_sub'],3,numsub), title=stn[istokes]+' subfreq {}'.format(i))
#         numsub += 1
# suptitle('Dust', y=1.02)


# figure()
# numsub = 1
# for i in range(d['nf_sub']):
#     for istokes in [0,1,2]:
#         hp.gnomview(maps[i,:,istokes], min=-rng[istokes], max=rng[istokes], rot=gal_center, reso=reso,
#                     sub=(d['nf_sub'],3,numsub), title=stn[istokes]+' subfreq {}'.format(i))
#         numsub += 1
# suptitle('CMB + Dust', y=1.02)



# Spectra

In [ ]:
### Pseudo-Spectra
# Spectra for the full sky
pnames = ['TT', 'EE', 'BB', 'TE']

lmax = 2*d['nside']
pixwin = hp.pixwin(d['nside'])[:lmax+1]

cl_cmb = np.zeros((d['nf_sub'], 2*d['nside']+1, 6))
cl_dust = np.zeros((d['nf_sub'], 2*d['nside']+1, 6))
cl = np.zeros((d['nf_sub'], 2*d['nside']+1, 6))
for i in range(d['nf_sub']): 
    cl_cmb[i,:,:] = hp.anafast(maps_cmb[i,:,:].T, lmax=lmax).T
    # cl_dust[i,:,:] = hp.anafast(maps_dust[i,:,:].T, lmax=lmax).T
    # cl[i,:,:] = hp.anafast(maps[i,:,:].T, lmax=lmax).T
ll = np.arange(2*d['nside']+1)

fact = ll * (ll+1) / 2 / np.pi
isp = 2
for i in range(d['nf_sub']):
    subplot(2,d['nf_sub'],i+1)
    plot(ll, fact * cl_cmb[i,:, isp] / np.sqrt(sky_frac), 'r', label='CMB - Cut')
    # plot(ll, fact * cl_dust[i,:, isp] / np.sqrt(sky_frac), 'b', label='Dust - Cut')
    # plot(ll, fact * cl[i,:, isp] / np.sqrt(sky_frac), 'g', label='CMB + Dust - Cut')
 
    plot(ll, fact * cl_cmb_full[i,:, isp], 'r', label='CMB - Full', alpha=0.5)
    # plot(ll, fact * cl_dust_full[i,:, isp], 'b', label='Dust - Full', alpha=0.5)
    # plot(ll, fact * cl_full[i,:, isp], 'g', label='CMB + Dust - Full', alpha=0.5)
    plot(ell, totDL[:,isp], label='Input CMB', alpha=0.5)


    yscale('log')
    ylim(1e-4,1e2)
    xlim(0,2*d['nside'])
    grid(alpha=0.4)
    xlabel('$\ell$')
    ylabel('$D_\ell$'+pnames[isp]+'({})'.format(i))
    title('Cut-Sky Band={}'.format(i))
    if i==0:
        legend()
tight_layout()



In [ ]:
from qubic import NamasterLib as nam

# Create a Namaster object
lmin = 25
lmax = 2 * d['nside'] - 1
delta_ell = 20

### Flat weighting
maskpix = np.zeros(12*d['nside']**2)
maskpix[okpix] = 1

Namaster = nam.Namaster(maskpix, lmin=lmin, lmax=lmax, delta_ell=delta_ell)
ell_binned, b = Namaster.get_binning(d['nside'])
#mask_apo = Namaster.get_apodized_mask()

hp.gnomview(Namaster.mask_apo, rot=gal_center, reso=20, title='Apodized Namaster Mask')
hp.graticule()

In [ ]:
plnames = ['TT', 'EE', 'BB', 'TE']

purify_b = False
purify_e = False
verbose = False
beam_correction = False
pixwin_correction = False

ncross = d['nf_sub'] * (d['nf_sub']-1)
w = None
xdl_cmb = np.zeros((d['nf_sub'], d['nf_sub'], len(ell_binned), 4))
xdl_dust = np.zeros((d['nf_sub'], d['nf_sub'], len(ell_binned), 4))
xdl = np.zeros((d['nf_sub'], d['nf_sub'], len(ell_binned), 4))
for i in range(d['nf_sub']):
    for j in range(i, d['nf_sub']):
        print(i,j)
        print('CMB')
        leff, xdl_cmb[i,j,:,:], w = Namaster.get_spectra(maps_cmb[i,:,:].T, map2=maps_cmb[j,:,:].T,
                                                    purify_e=purify_e, purify_b=purify_b, verbose=verbose,
                                                    beam_correction=beam_correction, pixwin_correction=pixwin_correction,
                                                    w=w)
        xdl_cmb[j,i,:,:] = xdl_cmb[i,j,:,:]

        # print('Dust')
        # leff, xdl_dust[i,j,:,:], w = Namaster.get_spectra(maps_dust[i,:,:].T, map2=maps_dust[j,:,:].T,
        #                                             purify_e=purify_e, purify_b=purify_b, verbose=verbose,
        #                                             beam_correction=beam_correction, pixwin_correction=pixwin_correction,
        #                                             w=w)
        # xdl_dust[j,i,:,:] = xdl_dust[i,j,:,:]

        # print('CMB + Dust')
        # leff, xdl[i,j,:,:], w = Namaster.get_spectra(maps[i,:,:].T, map2=maps[j,:,:].T,
        #                                             purify_e=purify_e, purify_b=purify_b, verbose=verbose,
        #                                             beam_correction=beam_correction, pixwin_correction=pixwin_correction,
        #                                             w=w)
        # xdl[j,i,:,:] = xdl[i,j,:,:]




In [ ]:
isp = 2
for i in range(d['nf_sub']):
    for j in range(i, d['nf_sub']):
        subplot(d['nf_sub'], d['nf_sub'],i+d['nf_sub']*j+1)
        errorbar(leff, xdl_cmb[i,j,:,isp],xerr=delta_ell/2, fmt='ro-', label='CMB')
        # errorbar(leff, xdl_dust[i,j,:,isp],xerr=delta_ell/2, fmt='bo-', label='Dust')
        # errorbar(leff, xdl[i,j,:,isp],xerr=delta_ell/2, fmt='go-', label='CMB + Dust')
        plot(ell, totDL[:,isp], label='Input CMB', alpha=0.5)


        if i == j:
            plot(ll, fact * cl_cmb[i,:, isp] / sky_frac, 'r', label='pseudo Cl CMB', alpha=0.5)
            # plot(ll, fact * cl_dust[i,:, isp] / sky_frac, 'b', label='pseudo Cl Dust', alpha=0.5)
            # plot(ll, fact * cl[i,:, isp] / sky_frac, 'g', label='pseudo Cl CMB + Dust', alpha=0.5)
 
        xlabel('$\ell$')
        ylabel('$D_\ell$'+plnames[isp]+'({},{})'.format(i,j))
        if ((i+j)==0):
            legend(fontsize=8, framealpha=0.1)
        yscale('log')
        ylim(1e-4,1e2)
        xlim(0,2*d['nside'])
        grid(alpha=0.4)